In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

['embeddings', 'train.csv', 'sample_submission.csv', 'test.csv']


In [2]:
import numpy as np
import pandas as pd
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.text import text_to_word_sequence
import os
from tqdm import tqdm
import re
from keras.preprocessing.sequence import pad_sequences
import tensorflow as tf
from datetime import datetime
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import f1_score
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
import itertools


tqdm.pandas('desc')



#Always seed the randomness of this universe.
np.random.seed(6)

Using TensorFlow backend.


In [3]:
#Reading csv's
train = pd.read_csv('../input/train.csv')
test = pd.read_csv('../input/test.csv')

In [4]:
# clean the numbers
def clean_numbers(x):
    x = re.sub('[0-9]{5,}', '#####', x)
    x = re.sub('[0-9]{4}', '####', x)
    x = re.sub('[0-9]{3}', '###', x)
    x = re.sub('[0-9]{2}', '##', x)
    return x

In [5]:
# clean text, separate the punction
puncts = [',', '.', '"', ':', ')', '(', '-', '!', '?', '|', ';', "'", '$', '&', '/', '[', ']', '>', '%', '=', '#', '*', '+', '\\', '•',  '~', '@', '£', 
 '·', '_', '{', '}', '©', '^', '®', '`',  '<', '→', '°', '€', '™', '›',  '♥', '←', '×', '§', '″', '′', 'Â', '█', '½', 'à', '…', 
 '“', '★', '”', '–', '●', 'â', '►', '−', '¢', '²', '¬', '░', '¶', '↑', '±', '¿', '▾', '═', '¦', '║', '―', '¥', '▓', '—', '‹', '─', 
 '▒', '：', '¼', '⊕', '▼', '▪', '†', '■', '’', '▀', '¨', '▄', '♫', '☆', 'é', '¯', '♦', '¤', '▲', 'è', '¸', '¾', 'Ã', '⋅', '‘', '∞', 
 '∙', '）', '↓', '、', '│', '（', '»', '，', '♪', '╩', '╚', '³', '・', '╦', '╣', '╔', '╗', '▬', '❤', 'ï', 'Ø', '¹', '≤', '‡', '√', ]

def clean_text(x):
    x = str(x)
    for punct in puncts:
        x = x.replace(punct, f' {punct} ')
    return x

In [6]:
# transfor the words into correct version
mispell_dict = {"ain't": "is not", "aren't": "are not","can't": "cannot", "'cause": "because", "could've": "could have", "couldn't": "could not", "didn't": "did not",  "doesn't": "does not", "don't": "do not", "hadn't": "had not", "hasn't": "has not", "haven't": "have not", "he'd": "he would","he'll": "he will", "he's": "he is", "how'd": "how did", "how'd'y": "how do you", "how'll": "how will", "how's": "how is",  "I'd": "I would", "I'd've": "I would have", "I'll": "I will", "I'll've": "I will have","I'm": "I am", "I've": "I have", "i'd": "i would", "i'd've": "i would have", "i'll": "i will",  "i'll've": "i will have","i'm": "i am", "i've": "i have", "isn't": "is not", "it'd": "it would", "it'd've": "it would have", "it'll": "it will", "it'll've": "it will have","it's": "it is", "let's": "let us", "ma'am": "madam", "mayn't": "may not", "might've": "might have","mightn't": "might not","mightn't've": "might not have", "must've": "must have", "mustn't": "must not", "mustn't've": "must not have", "needn't": "need not", "needn't've": "need not have","o'clock": "of the clock", "oughtn't": "ought not", "oughtn't've": "ought not have", "shan't": "shall not", "sha'n't": "shall not", "shan't've": "shall not have", "she'd": "she would", "she'd've": "she would have", "she'll": "she will", "she'll've": "she will have", "she's": "she is", "should've": "should have", "shouldn't": "should not", "shouldn't've": "should not have", "so've": "so have","so's": "so as", "this's": "this is","that'd": "that would", "that'd've": "that would have", "that's": "that is", "there'd": "there would", "there'd've": "there would have", "there's": "there is", "here's": "here is","they'd": "they would", "they'd've": "they would have", "they'll": "they will", "they'll've": "they will have", "they're": "they are", "they've": "they have", "to've": "to have", "wasn't": "was not", "we'd": "we would", "we'd've": "we would have", "we'll": "we will", "we'll've": "we will have", "we're": "we are", "we've": "we have", "weren't": "were not", "what'll": "what will", "what'll've": "what will have", "what're": "what are",  "what's": "what is", "what've": "what have", "when's": "when is", "when've": "when have", "where'd": "where did", "where's": "where is", "where've": "where have", "who'll": "who will", "who'll've": "who will have", "who's": "who is", "who've": "who have", "why's": "why is", "why've": "why have", "will've": "will have", "won't": "will not", "won't've": "will not have", "would've": "would have", "wouldn't": "would not", "wouldn't've": "would not have", "y'all": "you all", "y'all'd": "you all would","y'all'd've": "you all would have","y'all're": "you all are","y'all've": "you all have","you'd": "you would", "you'd've": "you would have", "you'll": "you will", "you'll've": "you will have", "you're": "you are", "you've": "you have", 'colour': 'color', 'centre': 'center', 'favourite': 'favorite', 'travelling': 'traveling', 'counselling': 'counseling', 'theatre': 'theater', 'cancelled': 'canceled', 'labour': 'labor', 'organisation': 'organization', 'wwii': 'world war 2', 'citicise': 'criticize', 'youtu ': 'youtube ', 'Qoura': 'Quora', 'sallary': 'salary', 'Whta': 'What', 'narcisist': 'narcissist', 'howdo': 'how do', 'whatare': 'what are', 'howcan': 'how can', 'howmuch': 'how much', 'howmany': 'how many', 'whydo': 'why do', 'doI': 'do I', 'theBest': 'the best', 'howdoes': 'how does', 'mastrubation': 'masturbation', 'mastrubate': 'masturbate', "mastrubating": 'masturbating', 'pennis': 'penis', 'Etherium': 'Ethereum', 'narcissit': 'narcissist', 'bigdata': 'big data', '2k17': '2017', '2k18': '2018', 'qouta': 'quota', 'exboyfriend': 'ex boyfriend', 'airhostess': 'air hostess', "whst": 'what', 'watsapp': 'whatsapp', 'demonitisation': 'demonetization', 'demonitization': 'demonetization', 'demonetisation': 'demonetization'}

def _get_mispell(mispell_dict):
    mispell_re = re.compile('(%s)' % '|'.join(mispell_dict.keys()))
    return mispell_dict, mispell_re

mispellings, mispellings_re = _get_mispell(mispell_dict)
def replace_typical_misspell(text):
    def replace(match):
        return mispellings[match.group(0)]
    return mispellings_re.sub(replace, text)

In [7]:
# Clean spellings
train["question_text"] = train["question_text"].progress_apply(lambda x: replace_typical_misspell(x))
test["question_text"] = test["question_text"].progress_apply(lambda x: replace_typical_misspell(x))

100%|██████████| 375806/375806 [00:09<00:00, 40021.45it/s]


In [8]:
# Clean the text
train["question_text"] = train["question_text"].progress_apply(lambda x: clean_text(x))
test["question_text"] = test["question_text"].progress_apply(lambda x: clean_text(x))

100%|██████████| 375806/375806 [00:15<00:00, 24435.60it/s]


In [9]:
# Clean numbers
train["question_text"] = train["question_text"].progress_apply(lambda x: clean_numbers(x))
test["question_text"] = test["question_text"].progress_apply(lambda x: clean_numbers(x))

100%|██████████| 375806/375806 [00:04<00:00, 84959.14it/s]


In [10]:
## fill up the missing values
train_X = train["question_text"].fillna("_##_").values
test_X = test["question_text"].fillna("_##_").values

In [11]:
MAX_WORD_TO_USE = 100000 # how many words to use in training
## Tokenize the sentences
tokenizer = Tokenizer(num_words=MAX_WORD_TO_USE)
tokenizer.fit_on_texts(list(train['question_text']) )
train_X = tokenizer.texts_to_sequences(train['question_text'])
test_X = tokenizer.texts_to_sequences(test['question_text'])

In [12]:
print(train_X[0])

[9, 48, 6520, 6990, 160, 57, 5977, 39, 4, 1132, 6, 1, 29]


In [13]:
## Pad the sentences 
MAX_LEN = 72 # number of time-steps.
train_X = pad_sequences(train_X, maxlen=MAX_LEN, truncating = 'post',padding='pre')
test_X = pad_sequences(test_X, maxlen=MAX_LEN, truncating = 'post',padding='pre')

In [14]:
print(train_X[0])

[   0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    9   48 6520 6990  160   57 5977   39    4 1132    6
    1   29]


In [15]:
# labels
train_y = train['target'].values

In [16]:
word_index = tokenizer.word_index
print(word_index['play'])


333


In [17]:
EMBED_SIZE = 300 
# load embedding file: glove
def load_glove(word_index):
    EMBEDDING_FILE = '../input/embeddings/glove.840B.300d/glove.840B.300d.txt'
    def get_coefs(word,*arr): 
        return word, np.asarray(arr, dtype='float32')[:300]
    embeddings_index = dict(get_coefs(*o.split(" ")) for o in open(EMBEDDING_FILE))

    # word_index = tokenizer.word_index
    nb_words = min(MAX_WORD_TO_USE, len(word_index))
    # some words might in word_index not in embeddings_index, so still need to initialize those words vector
    emb_mean,emb_std = 0.0, 1.0
    embedding_matrix = np.random.normal(emb_mean, emb_std, (nb_words, EMBED_SIZE))
    for word, i in tqdm(word_index.items()):
        if i >= MAX_WORD_TO_USE:
            continue
        embedding_vector = embeddings_index.get(word)
        # guarantee the word in word_index is in embeddings_index
        # if isn't, don't change the value
        if embedding_vector is not None:
            embedding_matrix[i] = embedding_vector
    return embedding_matrix 

In [18]:
def load_para(word_index):
    EMBEDDING_FILE = '../input/embeddings/paragram_300_sl999/paragram_300_sl999.txt'
    def get_coefs(word,*arr): 
        return word, np.asarray(arr, dtype='float32')
    embeddings_index = dict(get_coefs(*o.split(" ")) for o in open(EMBEDDING_FILE, encoding="utf8", errors='ignore') if len(o)>100)
    
    # word_index = tokenizer.word_index
    nb_words = min(MAX_WORD_TO_USE, len(word_index))
    emb_mean,emb_std = 0.0, 1.0
    embedding_matrix = np.random.normal(emb_mean, emb_std, (nb_words, EMBED_SIZE))
    for word, i in tqdm(word_index.items()):
        if i >= MAX_WORD_TO_USE: 
            continue
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None: 
            embedding_matrix[i] = embedding_vector
    
    return embedding_matrix

In [19]:
glove_embeddings = load_glove(word_index)
paragram_embeddings = load_para(word_index)

100%|██████████| 185823/185823 [00:00<00:00, 489323.66it/s]


In [20]:
# immerge two embeddings
embedding_matrix = np.mean([glove_embeddings, paragram_embeddings], axis=0)
embedding_matrix = embedding_matrix.astype('float32')

In [21]:
print(embedding_matrix.shape)

(100000, 300)


In [22]:
n_splits = 5 # how many folds
# StratifiedKFold provides train/test indices to split data in train/test sets for cross-validator.
# n_splits means how many train/test sets
splits = list(StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=6).split(train_X, train_y))

In [23]:
#Resetting the graph
tf.reset_default_graph()

#Seed the randomness
tf.set_random_seed(6)

In [24]:
numClasses = 2 # {Sincere,Insincere}
#Defining Placeholders
input_data = tf.placeholder(tf.int32, [None, MAX_LEN])
y_true = tf.placeholder(tf.float32, [None, numClasses])

In [25]:
# for dropout layer
hold_prob1 = tf.placeholder(tf.float32)

In [26]:
# Creating our Embedding matrix
data = tf.nn.embedding_lookup(embedding_matrix,input_data)

data = tf.transpose(data, [1, 0, 2])

Instructions for updating:
Colocations handled automatically by placer.


In [27]:
# For single layer LSTM
num_units = 256 # Number of nodes in hidden Layer
# kernel_initializer: starting value to initialize the weight.
LSTM_CELL = tf.contrib.cudnn_rnn.CudnnLSTM(num_layers=1, num_units=num_units, bias_initializer=tf.constant_initializer(0.1), 
                                           kernel_initializer=tf.contrib.layers.xavier_initializer())


For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.



In [28]:
value, _ = LSTM_CELL(inputs= data)

In [29]:
# Defining weights and biases for 1 st Fully Connected part of NN
nodes_on_FC = 64 # Number of nodes on FC layer
weight_fc1 = tf.Variable(tf.truncated_normal([num_units, nodes_on_FC]))
bias_fc1 = tf.Variable(tf.constant(0.1, shape=[nodes_on_FC]))

In [30]:
output = value[-1]

In [31]:
epsilon = 1e-4 # For batch normalization
# Defining 1st FC layer
y_pred_without_BN = tf.matmul(output, weight_fc1) + bias_fc1
# tf.nn.moments: calculating batch_mean and batch_variance
batch_mean, batch_var = tf.nn.moments(y_pred_without_BN,[0])
# Creating parameters for Batch normalization
scale = tf.Variable(tf.ones([nodes_on_FC]))
beta = tf.Variable(tf.zeros([nodes_on_FC]))
#Implementing batch normalization
y_pred_without_activation = tf.nn.batch_normalization(y_pred_without_BN,batch_mean,batch_var,beta,scale,epsilon)

In [32]:
#Applying RELU
y_pred_with_activation = tf.nn.relu(y_pred_without_activation)
#Dropout Layer 1
y_pred_with_dropout = tf.nn.dropout(y_pred_with_activation,rate=1-hold_prob1)

In [33]:
#Defining weights and biases for output layer
weight_output_layer = tf.Variable(tf.truncated_normal([nodes_on_FC, numClasses]))
bias_output_layer = tf.Variable(tf.constant(0.1, shape=[numClasses]))
# Calculating last layer of NN
y_pred = tf.matmul(y_pred_with_dropout, weight_output_layer) + bias_output_layer
# activation
y_pred_softmax = tf.nn.softmax(y_pred)


In [34]:
# Defining Accuracy
# y_true: [None, numClasses]
matches = tf.equal(tf.argmax(y_pred,1),tf.argmax(y_true,1))
acc = tf.reduce_mean(tf.cast(matches,tf.float32))

In [35]:
# Defining Loss Function
cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels=y_true,logits=y_pred))
# Defining objective
training = tf.train.RMSPropOptimizer(learning_rate=0.001).minimize(cross_entropy)

Instructions for updating:
Use tf.cast instead.


In [36]:
##Initializing trainable/non-trainable variables
init = tf.global_variables_initializer()
print("Model was built up")

Model was built up


In [37]:
#Defining a function for early stopping
def early_stopping_check(x):
    if np.mean(x[-20:]) <= np.mean(x[-80:]):
        return True
    else:
        return False

In [38]:
train_preds = np.zeros((len(train_X),2))
test_preds = []
#GPU settings
config = tf.ConfigProto(allow_soft_placement=True)
config.gpu_options.allow_growth = True
config.gpu_options.allocator_type = 'BFC'

In [39]:
iterations = 5000 # How many iterations to train
batchSize = 256 # how many samples to feed neural network
# Opening up Session
for j, (train_idx, valid_idx) in enumerate(splits): 
    print( "Fold {} has started".format(j) )
    with tf.Session(config=config) as sess:
        #Running init
        sess.run(init) 
        #Creating a list for Early Stopping
        val_scores_loss= []
        val_scores_f1= []
        #Main loop
        for i in range(iterations):
            random_numbers = np.random.choice(train_idx,batchSize)
            #  pd.get_dummies: one hot encode
            _,c = sess.run([training,cross_entropy] ,feed_dict = {input_data: train_X[random_numbers], y_true: pd.get_dummies(train_y[random_numbers]).values, 
                                                                  hold_prob1: 0.6} )
            # Validating Loop
            if i % 50 == 0:
                random_numbers_cv = np.random.choice(valid_idx,batchSize)
                
                # Getting train stats.
                acc_tr,loss_tr,prob_tr = sess.run([acc,cross_entropy,y_pred],feed_dict={input_data:train_X[random_numbers],y_true:pd.get_dummies(train_y[random_numbers]).values, hold_prob1:1.0 })
                # Train F1
                pred_tr = prob_tr.argmax(axis=1)
                # f1_score: 2*precision*recall/(precision+recall)
                f1_tr = f1_score(train_y[random_numbers],pred_tr)
                                
                # Getting validation stats.
                acc_cv,loss_cv,prob_cv = sess.run([acc,cross_entropy,y_pred],feed_dict = {input_data:train_X[random_numbers_cv],y_true:pd.get_dummies(train_y[random_numbers_cv]).values,hold_prob1:1.0})
                
                pred_cv = prob_cv.argmax(axis=1)
                 
                f1_cv = f1_score(train_y[random_numbers_cv],pred_cv)

                #Appending loss_cv to val_scores:
                val_scores_loss.append(loss_cv)
                val_scores_f1.append(f1_cv)
                
                #Printing on each 1000 iterations
                if i%1000 ==0:
                    print("Training  : Iter = {}, Train Loss = {}, Train Accuracy = {}".format(i,loss_tr,acc_tr))
                    print("Validation: Iter = {}, CV    Loss = {}, CV Accuracy = {}".format(i,loss_cv,acc_cv))
                    
                    print("Training  : Iter = {}, Train-F1 = {}".format(i,f1_tr))
                    print("Validation: Iter = {}, CV-F1    = {}".format(i,f1_cv))
                    print("-----------------------------")

                    #If validation loss didn't decrease for val_loop_iter * 20 iters, stop.
                    """if early_stopping_check(val_scores_loss) == False:
                        saver.save(sess, os.path.join(os.getcwd(),"1_layered_GRU.ckpt"),global_step=i)
                        continue"""
                if (np.mean(val_scores_f1[-5:]) > 0.7000) & ( np.mean(val_scores_loss[-5:]) <0.0800 ):
                    print("Early Finishing")
                    print( val_scores_f1[-5:] )
                    print( val_scores_loss[-5:] )
                    break
        # Training_validation predictions        
        for r in range(0,len(valid_idx),batchSize):
            index_train = valid_idx[r:r + batchSize]
            train_preds[index_train,:] = y_pred_softmax.eval(feed_dict={input_data:train_X[index_train],hold_prob1:1.0},session=sess)

        # Test predictions        
        test_fold = []
        for r in range(0,len(test_X),batchSize):
            k = y_pred_softmax.eval(feed_dict={input_data:test_X[r:r+batchSize],hold_prob1:1.0},session=sess)
            test_fold.append(k)
            del k
        test_preds.append(test_fold)
        print("Training {} has finished".format(j))


Fold 0 has started
Training  : Iter = 0, Train Loss = 1.3672481775283813, Train Accuracy = 0.6953125
Validation: Iter = 0, CV    Loss = 1.3367371559143066, CV Accuracy = 0.703125
Training  : Iter = 0, Train-F1 = 0.07142857142857144
Validation: Iter = 0, CV-F1    = 0.13636363636363638
-----------------------------


/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


Training  : Iter = 1000, Train Loss = 0.10078371316194534, Train Accuracy = 0.953125
Validation: Iter = 1000, CV    Loss = 0.141015887260437, CV Accuracy = 0.91796875
Training  : Iter = 1000, Train-F1 = 0.6666666666666667
Validation: Iter = 1000, CV-F1    = 0.43243243243243246
-----------------------------
Training  : Iter = 2000, Train Loss = 0.07479945570230484, Train Accuracy = 0.96875
Validation: Iter = 2000, CV    Loss = 0.13169264793395996, CV Accuracy = 0.953125
Training  : Iter = 2000, Train-F1 = 0.6923076923076924
Validation: Iter = 2000, CV-F1    = 0.4545454545454545
-----------------------------
Training  : Iter = 3000, Train Loss = 0.10452912002801895, Train Accuracy = 0.953125
Validation: Iter = 3000, CV    Loss = 0.10948401689529419, CV Accuracy = 0.94140625
Training  : Iter = 3000, Train-F1 = 0.4
Validation: Iter = 3000, CV-F1    = 0.2105263157894737
-----------------------------
Training  : Iter = 4000, Train Loss = 0.09429159760475159, Train Accuracy = 0.96875
Validati

In [40]:
print(train_preds.shape)
train_preds_0_values = train_preds[:, 0]
print(train_preds_0_values.shape)

(1306122, 2)
(1306122,)


In [41]:
print(train_preds[:1])

[[0.93453211 0.06546786]]


In [42]:
def bestThresshold(y_train,train_preds2):
    tmp = [0,0,0] # delta value, current f1_score, max f1_score
    delta = 0
    for tmp[0] in tqdm(np.arange(0.1, 0.801, 0.01)):
        tmp[1] = f1_score(y_train, (np.array(train_preds2)<=tmp[0]).astype(int))
        if tmp[1] > tmp[2]:
            delta = tmp[0]
            tmp[2] = tmp[1]
    print('best threshold is {:.4f} with F1 score: {:.4f}'.format(delta, tmp[2]))
    return delta

In [43]:
delta = bestThresshold(train_y,train_preds_0_values)

100%|██████████| 71/71 [00:16<00:00,  4.37it/s]

best threshold is 0.6700 with F1 score: 0.6509


In [44]:
print(((test_preds[0])[0])[0])

[0.18328877 0.81671125]


In [45]:
our_preds_test = []
for i in test_preds:
    l = np.zeros((0,2))
    for j in i:
        l = np.append(l,j,axis=0)
    our_preds_test.append(l)

In [46]:
test_predictions = our_preds_test[0] + our_preds_test[1] + our_preds_test[2] + our_preds_test[3] + our_preds_test[4]
test_predictions = test_predictions / 5
test_predictions = test_predictions[:,0]

In [47]:
submission1 = test[['qid']].copy()
submission1['prediction'] = (test_predictions <= delta).astype(int)

In [48]:
print(submission1['prediction'].values[:6])

[1 0 0 0 0 0]


In [49]:
submission1.to_csv('submission.csv', index=False)